In [2]:
import scipy as sc
import numpy as np

In [40]:
#Matriz de adjacencia que modela o grafo cujo os vértices são as opiniões/posts
A =[[0, 1, 1, 1],
    [1, 0, 1, 0],
    [1, 0, 0, 1],
    [1, 1, 1, 0]]

#PARAMETROS INICIAIS DO PROBLEMA

B = 0.67 #Taxa de transmissão

#Probabilidade de um post influenciar um vértice i num tempo t+1
def probx(tempo, MA, i, Taxainfo): 
    prod = 1
    x = [sc.stats.norm.pdf(t, 0, 1), sc.stats.poisson.pmf(t, 1), sc.stats.bernoulli.pmf(t, 1, 1), sc.stats. poisson.pmf(t, 1)] #vetor de estado
    xi = np.array(x)
    xi.transpose
    for j in range(len(A)-1):
        prod *= (1 - B*A[i][j]*xi[j]) #produtorio
    return prod

#Calculo de quanto tempo cada vertice leva para ser influenciado
for i in range(len(A)):
    
    t = 0 #Parâmetros de informação
    k = 0 #Contador de iteração
    while prob != 1:
        prob = probx(t, A, i, B)
        #print(prob)
        if prob > 0.9999999999:
                prob = 1
        
        if prob == 1:
            print(f'vértice {i+1} foi convencido por t após {k} tempos')
            prob = 0
            break
        #else:
            print(f'vértice {i+1} sucetivel a t com probabilidade atual de {prob} no tempo {t}')
        
        t = t + 1
        k = k+1
        prob = 0


vértice 1 foi convencido por t após 13 tempos
vértice 2 foi convencido por t após 7 tempos
vértice 3 foi convencido por t após 7 tempos
vértice 4 foi convencido por t após 13 tempos
